In [1]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from dotenv import load_dotenv
import os
from database_utils import calculate_embedding
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Qdrant
from qdrant_client.http.models import Distance, VectorParams
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import json
import warnings

warnings.filterwarnings("ignore")


load_dotenv()
QDRANT_KEY = os.getenv('QDRANT_KEY')
QDRANT_CLUSTER_URL = os.getenv('QDRANT_CLUSTER_URL')
HUGGING_FACE_TOKEN = os.getenv('HUGGING_FACE_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Search for related articles

In [2]:
def query_qdrant(question, collection_name, qdrant_url, qdrant_key, top_k=5):
    """
    Query Qdrant database with a question to find the most similar records.

    Parameters:
    - question (str): The question or query string.
    - collection_name (str): The name of the Qdrant collection to query.
    - qdrant_url (str): The Qdrant server URL.
    - qdrant_key (str): API key for Qdrant authentication.
    - top_k (int): Number of top results to return (default is 5).

    Returns:
    - list: A list of the top-k results with metadata and similarity scores.
    """
    # Initialize Qdrant client
    client = QdrantClient(url=qdrant_url, api_key=qdrant_key)

    # Load a SentenceTransformer model for embedding generation
    model = FastEmbedEmbeddings()  # Replace with your model if needed

    # Generate embedding for the question
    question_embedding = calculate_embedding(question)

    # Perform the search in the specified collection
    search_results = client.search(
        collection_name=collection_name,
        query_vector=question_embedding,
        limit=top_k,  # Return top-k results
    )

    # Parse and return the results
    results = []
    for result in search_results:
        results.append({
            "id": result.id,
            "score": result.score,
            "payload": result.payload,  # Metadata of the record
        })

    return results

In [3]:
results = query_qdrant("Give me articles about Steve Jobs", "bbc_news_articles", QDRANT_CLUSTER_URL, QDRANT_KEY)
print(json.dumps(results, indent=4))

[
    {
        "id": "96efba6b-b4c2-43bf-a4c3-c4f82debfb27",
        "score": 0.6449732,
        "payload": {
            "page_content": "Apple makes blogs reveal sources  Apple has won its legal fight to make three bloggers reveal who told them about unreleased products.  The bid to unmask the employees leaking information was launched in December 2004 following online articles about Apple's Asteroid product. Now Apple has won the right to see e-mail records from the three bloggers to root out the culprit. A lawyer for the three bloggers said the ruling set a dangerous precedent that could harm all news reporters.  Apple's lawsuit accused anonymous people of stealing trade secrets about the Asteroid music product and leaking them to the PowerPage, Apple Insider and Think Secret websites. All three are Apple fan sites that obsessively watch the iconic firm for information about future products. Apple is notoriously secretive about upcoming products which gives any snippets of informa

# Connect asking questions with database search

In [4]:
client = QdrantClient(url=QDRANT_CLUSTER_URL, api_key=QDRANT_KEY)

vector_store = Qdrant(
    client=client,
    collection_name="bbc_news_articles",
    embeddings=FastEmbedEmbeddings() ,
)

llm = ChatOpenAI(model_name= "gpt-3.5-turbo", temperature=0)

In [5]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vector_store.as_retriever()
)
result = qa_chain({"query": "How much ipod with one gigabyte storage costs that was unveiled by Steve Jobs during annual MacWorld speech?"})
result["result"]

'The iPod with one gigabyte of storage that was unveiled by Steve Jobs during the annual MacWorld speech costs $149 in the US and £99 in the UK.'

In [6]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vector_store.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr({"query": "How much ipod with one gigabyte storage costs that was unveiled by Steve Jobs during annual MacWorld speech?"})
result["result"]

"I'm sorry, but the text provided does not mention the specific cost of an iPod with one gigabyte storage that was unveiled by Steve Jobs during an annual MacWorld speech. If you have any other questions or need further assistance, please let me know."

# Build a prompt

In [7]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use 5 sentences maximum. Keep the answer as concise as possible. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question", "n_sentences"], template=template)

question =  "How much ipod with one gigabyte storage costs that was unveiled by Steve Jobs during annual MacWorld speech?"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vector_store.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

'The iPod with one gigabyte of storage that was unveiled by Steve Jobs during the annual MacWorld speech costs $149 (£99 in the UK).'

# Add memory

In [8]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
retriever=vector_store.as_retriever()


qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [9]:
question = "How much ipod with one gigabyte storage costs that was unveiled by Steve Jobs during annual MacWorld speech?"
result = qa({"question": question})
result['answer']

'The iPod shuffle with one gigabyte of storage costs $149 (£99) as unveiled by Steve Jobs during the annual MacWorld speech.'

In [10]:
question = "Is there a cheaper option?"
result = qa({"question": question})
result['answer']

'Yes, Apple has released the iPod shuffle, which is a cheaper option compared to other iPod models. The iPod shuffle uses cheaper flash memory rather than hard drives and is available in two versions, one offering 512MB of storage for $99 (£69 in the UK) and a second with one gigabyte of storage for $149 (£99).'